## __Check first before starting__

In [1]:
import os

Working_directory = os.path.normpath("/mnt/mydisk/Continual_Learning_JL/Continual_Learning/")
os.chdir(Working_directory)
print(f"Working directory: {os.getcwd()}")

Working directory: /mnt/mydisk/Continual_Learning_JL/Continual_Learning


## __All imports__

In [2]:
# Operating system and file management
import os
import shutil
import contextlib
import traceback
import gc
import copy
from collections import defaultdict
import subprocess
import time
import re, pickle
import scipy.io
from scipy.io import loadmat
from glob import glob
from math import ceil

# Jupyter notebook widgets and display
import ipywidgets as widgets
from IPython.display import display

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Plotting and visualization
import matplotlib.pyplot as plt
from mpl_interactions import zoom_factory, panhandler

# Machine learning and preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pickle
from ta import trend, momentum, volatility, volume

# Mathematical and scientific computing
import math
from scipy.ndimage import gaussian_filter1d

# Type hinting
from typing import Callable, Tuple

# Deep learning with PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision.models import resnet18
from sklearn.utils.class_weight import compute_class_weight

## __📁 Path Settings and Constants__
This cell defines essential paths and constants for the CPSC2018 ECG dataset processing:
- `BASE_DIR`: Root directory of the project.
- `save_dir`: Path to the preprocessed `.npy` files (one for each continual learning period).
- `ECG_PATH`: Directory containing original `.mat` and `.hea` files.
- `MAX_LEN`: Length of each ECG sample, fixed to 5000 time steps (i.e., 10 seconds at 500Hz).

In [3]:
BASE_DIR = "/mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL"
save_dir = os.path.join(BASE_DIR, "processed")
ECG_PATH = os.path.join(BASE_DIR, "datas")
MAX_LEN = 5000

## __🏷️ Label Mapping and Period Configuration__

This section defines:
- `snomed_map`: Mapping from SNOMED CT codes to readable class names for 9 major ECG conditions.
- `period_label_map`: Incremental learning task structure across four periods.  
  Class `1` is reserved for "OTHER" abnormalities until Period 4 when all 9 classes are explicitly categorized.
- `print_class_distribution()`: Helper function to show class-wise data distribution.


In [4]:
# SNOMED CT to readable names
snomed_map = {
    "426783006": "NSR",    # 正常竇性心律
    "270492004": "I-AVB",  # 一度房室傳導阻滯
    "164889003": "AF",     # 心房纖維顫動
    "164909002": "LBBB",   # 左束支傳導阻滯
    "59118001":  "RBBB",   # 右束支傳導阻滯
    "284470004": "PAC",    # 心房早期搏動
    "164884008": "PVC",    # 室性早期搏動
    "429622005": "STD",    # ST 段壓低
    "164931005": "STE"     # ST 段抬高
}

# Period class mapping (固定 class 1 是「其他異常」直到 P4 移除)
period_label_map = {
    1: {"NSR": 0, "OTHER": 1},
    2: {"NSR": 0, "I-AVB": 2, "AF": 3, "OTHER": 1},
    3: {"NSR": 0, "I-AVB": 2, "AF": 3, "LBBB": 4, "RBBB": 5, "OTHER": 1},
    4: {"NSR": 0, "I-AVB": 2, "AF": 3, "LBBB": 4, "RBBB": 5, "PAC": 6, "PVC": 7, "STD": 8, "STE": 9}
}

def print_class_distribution(y, label_map):
    y = np.array(y).flatten()
    total = len(y)
    all_labels = sorted(label_map.values())
    print("\n📊 Class Distribution")
    for lbl in all_labels:
        count = np.sum(y == lbl)
        label = [k for k, v in label_map.items() if v == lbl]
        name = label[0] if label else str(lbl)
        print(f"  ├─ Label {lbl:<2} ({name:<10}) → {count:>5} samples ({(count/total)*100:5.2f}%)")

def ensure_folder(folder_path: str) -> None:
    """Ensure the given folder exists, create it if not."""
    os.makedirs(folder_path, exist_ok=True)


## 📦 EX. Load Example (Period 4) Data and View Format

This example demonstrates how to load preprocessed `.npy` data for **Period 4**, and inspect the dataset shapes and label distribution.  
Use this format as a reference when loading data in other methods (e.g., EWC, PNN, DynEx-CLoRA).

Each ECG sample:
- Has shape `(5000, 12)` → represents 10 seconds (at 500Hz) across 12-lead channels.
- Corresponding label is an integer ID (e.g., 0–9) defined by `period_label_map[4]`.

In [5]:
# # 範例:載入 period 4
# save_dir = os.path.join(BASE_DIR, "processed")
# X_train = np.load(os.path.join(save_dir, "X_train_p4.npy"))
# y_train = np.load(os.path.join(save_dir, "y_train_p4.npy"))
# X_test = np.load(os.path.join(save_dir, "X_test_p4.npy"))
# y_test = np.load(os.path.join(save_dir, "y_test_p4.npy"))

# print("✅ Loaded")
# print("X_train shape:", X_train.shape)
# print("y_train shape:", y_train.shape)
# print("X_test shape:", X_test.shape)
# print("y_test shape:", y_test.shape)
# print_class_distribution(y_train, period_label_map[4])
# print_class_distribution(y_test, period_label_map[4])

# del X_train, y_train, X_test, y_test


## __Check GPU, CUDA, Pytorch__

In [6]:
!nvidia-smi

Sat May 10 15:56:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.07             Driver Version: 570.133.07     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               Off |   00000000:2A:00.0 Off |                  Off |
| 30%   47C    P2             80W /  300W |    3281MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### CUDA Details

In [7]:
def check_gpu_config():
    """
    Check GPU availability and display detailed configuration information.
    """
    # Check if GPU is available
    gpu_available = torch.cuda.is_available()
    
    # Print header
    print("=" * 50)
    print("GPU Configuration Check".center(50))
    print("=" * 50)
    
    # Basic GPU availability
    print(f"{'PyTorch Version':<25}: {torch.__version__}")
    print(f"{'GPU Available':<25}: {'Yes' if gpu_available else 'No'}")
    
    # If GPU is available, print detailed info
    if gpu_available:
        print("-" * 50)
        print("GPU Details".center(50))
        print("-" * 50)
        
        # Device info
        print(f"{'Device Name':<25}: {torch.cuda.get_device_name(0)}")
        print(f"{'Number of GPUs':<25}: {torch.cuda.device_count()}")
        print(f"{'Current Device Index':<25}: {torch.cuda.current_device()}")
        
        # Compute capability and CUDA cores
        props = torch.cuda.get_device_properties(0)
        print(f"{'Compute Capability':<25}: {props.major}.{props.minor}")
        print(f"{'Total CUDA Cores':<25}: {props.multi_processor_count * 128}")  # Approx. 128 cores per SM
        
        # Memory info
        total_memory = props.total_memory / (1024 ** 3)  # Convert to GB
        memory_allocated = torch.cuda.memory_allocated(0) / (1024 ** 3)
        memory_reserved = torch.cuda.memory_reserved(0) / (1024 ** 3)
        print(f"{'Total Memory (GB)':<25}: {total_memory:.2f}")
        print(f"{'Allocated Memory (GB)':<25}: {memory_allocated:.2f}")
        print(f"{'Reserved Memory (GB)':<25}: {memory_reserved:.2f}")
    else:
        print("-" * 50)
        print("No GPU detected. Running on CPU.".center(50))
        print("-" * 50)
    
    print("=" * 50)

if __name__ == "__main__":
    check_gpu_config()

             GPU Configuration Check              
PyTorch Version          : 2.5.1
GPU Available            : Yes
--------------------------------------------------
                   GPU Details                    
--------------------------------------------------
Device Name              : NVIDIA RTX A6000
Number of GPUs           : 3
Current Device Index     : 0
Compute Capability       : 8.6
Total CUDA Cores         : 10752
Total Memory (GB)        : 47.41
Allocated Memory (GB)    : 0.00
Reserved Memory (GB)     : 0.00


### PyTorch Details

In [8]:
def print_torch_config():
    """Print PyTorch and CUDA configuration in a formatted manner."""
    print("=" * 50)
    print("PyTorch Configuration".center(50))
    print("=" * 50)
    
    # Basic PyTorch and CUDA info
    print(f"{'PyTorch Version':<25}: {torch.__version__}")
    print(f"{'CUDA Compiled Version':<25}: {torch.version.cuda}")
    print(f"{'CUDA Available':<25}: {'Yes' if torch.cuda.is_available() else 'No'}")
    print(f"{'Number of GPUs':<25}: {torch.cuda.device_count()}")

    # GPU details if available
    if torch.cuda.is_available():
        print(f"{'GPU Name':<25}: {torch.cuda.get_device_name(0)}")

    print("-" * 50)
    
    # Seed setting
    torch.manual_seed(42)
    print(f"{'Random Seed':<25}: 42 (Seeding successful!)")
    
    print("=" * 50)

if __name__ == "__main__":
    print_torch_config()

              PyTorch Configuration               
PyTorch Version          : 2.5.1
CUDA Compiled Version    : 12.1
CUDA Available           : Yes
Number of GPUs           : 3
GPU Name                 : NVIDIA RTX A6000
--------------------------------------------------
Random Seed              : 42 (Seeding successful!)


## __⚙️ GPU Selection — Auto-select the least loaded GPU__
This code automatically scans available GPUs and selects the one with the lowest current memory usage.


In [9]:
def auto_select_cuda_device(verbose=True):
    """
    Automatically selects the CUDA GPU with the least memory usage.
    Falls back to CPU if no GPU is available.
    """
    if not torch.cuda.is_available():
        print("🚫 No CUDA GPU available. Using CPU.")
        return torch.device("cpu")

    try:
        # Run nvidia-smi to get memory usage of each GPU
        smi_output = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'],
            encoding='utf-8'
        )
        memory_used = [int(x) for x in smi_output.strip().split('\n')]
        best_gpu = int(np.argmin(memory_used))

        if verbose:
            print("🎯 Automatically selected GPU:")
            print(f"    - CUDA Device ID : {best_gpu}")
            print(f"    - Memory Used    : {memory_used[best_gpu]} MiB")
            print(f"    - Device Name    : {torch.cuda.get_device_name(best_gpu)}")
        return torch.device(f"cuda:{best_gpu}")
    except Exception as e:
        print(f"⚠️ Failed to auto-detect GPU. Falling back to cuda:0. ({e})")
        return torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Execute and assign
device = auto_select_cuda_device()

🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 3281 MiB
    - Device Name    : NVIDIA RTX A6000


## __Model__

### ResNet 18 - 1D (ResNet18_1D_big_inplane)

In [10]:
class LoRAConv1d(nn.Module):
    def __init__(self, conv_layer: nn.Conv1d, rank: int):
        super(LoRAConv1d, self).__init__()
        self.conv = conv_layer
        self.rank = rank
        # 為適應LoRA低秩分解創建A和B矩陣
        self.lora_A = nn.Parameter(torch.zeros(conv_layer.out_channels, rank))
        self.lora_B = nn.Parameter(torch.zeros(rank, conv_layer.in_channels * conv_layer.kernel_size[0]))
        # 初始化權重：A用正態分佈，B用零初始化以確保訓練開始時LoRA無影響
        nn.init.normal_(self.lora_A, mean=0.0, std=0.01)
        nn.init.zeros_(self.lora_B)

    def forward(self, x):
        # 計算LoRA權重並重塑為卷積核形狀
        lora_weight = torch.matmul(self.lora_A, self.lora_B).view(
            self.conv.out_channels, self.conv.in_channels, self.conv.kernel_size[0]
        )
        # 將LoRA權重添加到原始權重
        adapted_weight = self.conv.weight + lora_weight
        # 使用修改後的權重執行卷積
        return F.conv1d(x, adapted_weight, bias=self.conv.bias,
                        stride=self.conv.stride, padding=self.conv.padding,
                        dilation=self.conv.dilation, groups=self.conv.groups)
                        
    def parameters(self, recurse=True):
        # 只返回LoRA參數，不包括原始卷積層參數
        return [self.lora_A, self.lora_B]

class BasicBlock1d_LoRA(nn.Module):
    expansion = 1
    def __init__(self, inplanes, planes, stride=1, downsample=None, lora_rank=None):
        super(BasicBlock1d_LoRA, self).__init__()
        self.conv1 = nn.Conv1d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv1d(planes, planes, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample
        self.stride = stride

        self.lora_rank = lora_rank
        self.lora_adapter = None  # 延遲初始化LoRA適配器

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        if self.lora_adapter is not None:
            # 如果LoRA適配器存在，使用它處理
            out = self.lora_adapter(out)
        else:
            # 否則使用原始conv2
            out = self.conv2(out)

        out = self.bn2(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

class ResNet18_1D_LoRA(nn.Module):
    def __init__(self, input_channels=12, output_size=9, inplanes=64, lora_rank=4):
        super(ResNet18_1D_LoRA, self).__init__()
        self.inplanes = inplanes
        self.lora_rank = lora_rank

        # 初始卷積層
        self.conv1 = nn.Conv1d(input_channels, inplanes, kernel_size=15, stride=2, padding=7, bias=False)
        self.bn1 = nn.BatchNorm1d(inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        # 殘差層
        self.layer1 = self._make_layer(BasicBlock1d_LoRA, 64, 2)
        self.layer2 = self._make_layer(BasicBlock1d_LoRA, 128, 2, stride=2)
        self.layer3 = self._make_layer(BasicBlock1d_LoRA, 256, 2, stride=2)
        self.layer4 = self._make_layer(BasicBlock1d_LoRA, 512, 2, stride=2)

        # 自適應池化（平均和最大）
        self.adaptiveavgpool = nn.AdaptiveAvgPool1d(1)
        self.adaptivemaxpool = nn.AdaptiveMaxPool1d(1)

        # 全連接層與dropout
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(512 * 2, output_size)  # *2因為concat了avg和max池化

        # 初始化權重
        self.init_weights()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.lora_rank))
        self.inplanes = planes
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, lora_rank=self.lora_rank))
        return nn.Sequential(*layers)

    def forward(self, x):
        # 預期輸入形狀: (batch_size, time_steps, channels)
        x = x.permute(0, 2, 1)  # → (batch_size, channels, time_steps)
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        # 應用平均和最大池化
        x1 = self.adaptiveavgpool(x)
        x2 = self.adaptivemaxpool(x)
        
        # 連接池化結果
        x = torch.cat((x1, x2), dim=1)
        
        # 展平
        x = x.view(x.size(0), -1)
        
        # 應用dropout
        x = self.dropout(x)
        
        # 最終分類
        x = self.fc(x)
        
        return x

    def init_weights(self):
        """初始化網絡權重"""
        for m in self.modules():
            if isinstance(m, (nn.Linear, nn.Conv1d)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def init_lora(self):
        """Period 2開始呼叫一次，初始化並固定使用LoRA適配器"""
        lora_count = 0
        for module in self.modules():
            if isinstance(module, BasicBlock1d_LoRA) and module.lora_adapter is None:
                # 為每個BasicBlock的conv2層創建LoRA適配器
                module.lora_adapter = LoRAConv1d(module.conv2, self.lora_rank).to(next(self.parameters()).device)
                lora_count += 1
        print(f"✅ LoRA adapters initialized for {lora_count} conv2 layers")

    def get_trainable_parameters(self):
        """返回可訓練參數列表（用於優化器）並提供參數統計"""
        lora_params = []
        lora_names = []
        fc_params = []
        fc_names = []
        
        # 計算總參數數量
        total_params = sum(p.numel() for p in self.parameters())
        
        # 收集所有LoRA參數
        # for name, module in self.named_modules():
        #     if isinstance(module, LoRAConv1d):
        #         for param_name, param in module.named_parameters():
        #             lora_params.append(param)
        #             lora_names.append(f"{name}.{param_name}")
        
        for name, module in self.named_modules():
            if isinstance(module, LoRAConv1d):
                lora_params.append(module.lora_A)
                lora_names.append(f"{name}.lora_A")
                lora_params.append(module.lora_B)
                lora_names.append(f"{name}.lora_B")

        # 添加fc層參數
        for name, param in self.fc.named_parameters():
            fc_params.append(param)
            fc_names.append(f"fc.{name}")
        
        # 計算統計數據
        trainable_params = lora_params + fc_params
        frozen_params = total_params - sum(p.numel() for p in trainable_params)
        lora_param_count = sum(p.numel() for p in lora_params)
        fc_param_count = sum(p.numel() for p in fc_params)
        trainable_param_count = lora_param_count + fc_param_count
        
        # 打印統計信息
        print(f"📊 Parameter Statistics:")
        print(f"  - Total parameters: {total_params:,}")
        print(f"  - Trainable parameters: {trainable_param_count:,} ({trainable_param_count/total_params*100:.2f}%)")
        print(f"    - LoRA parameters: {lora_param_count:,} ({lora_param_count/total_params*100:.2f}%)")
        print(f"    - FC parameters: {fc_param_count:,} ({fc_param_count/total_params*100:.2f}%)")
        print(f"  - Frozen parameters: {frozen_params:,} ({frozen_params/total_params*100:.2f}%)")
        
        print(f"🧠 Trainable parameter names:")
        for name in lora_names:
            print(f"  ✅ {name} (LoRA)")
        for name in fc_names:
            print(f"  ✅ {name} (FC)")
        
        return trainable_params

## __Training and validation function__

### Extra Function

In [11]:
def compute_classwise_accuracy(student_logits_flat, y_batch, class_correct, class_total):
    """
    Computes per-class accuracy by accumulating correct and total samples for each class using vectorized operations.
    
    Args:
        student_logits_flat (torch.Tensor): Model predictions (logits) in shape [batch_size * seq_len, output_size]
        y_batch (torch.Tensor): True labels in shape [batch_size * seq_len]
        class_correct (dict): Dictionary to store correct predictions per class
        class_total (dict): Dictionary to store total samples per class
    """
    # Ensure inputs are on the same device
    if student_logits_flat.device != y_batch.device:
        raise ValueError("student_logits_flat and y_batch must be on the same device")

    # Convert logits to predicted class indices
    predictions = torch.argmax(student_logits_flat, dim=-1)  # Shape: [batch_size * seq_len]

    # Compute correct predictions mask
    correct_mask = (predictions == y_batch)  # Shape: [batch_size * seq_len], boolean

    # Get unique labels in this batch
    unique_labels = torch.unique(y_batch)

    # Update class_total and class_correct using vectorized operations
    for label in unique_labels:
        label = label.item()  # Convert tensor to scalar
        if label not in class_total:
            class_total[label] = 0
            class_correct[label] = 0
        
        # Count total samples for this label
        label_mask = (y_batch == label)
        class_total[label] += label_mask.sum().item()
        
        # Count correct predictions for this label
        class_correct[label] += (label_mask & correct_mask).sum().item()

In [12]:
def get_model_parameter_info(model):
    total_params = sum(p.numel() for p in model.parameters())
    param_size_bytes = total_params * 4
    param_size_MB = param_size_bytes / (1024**2)
    return total_params, param_size_MB

In [13]:
def compute_class_weights(y: np.ndarray, num_classes: int, exclude_classes: list = None) -> torch.Tensor:
    """
    計算 class weights（inverse frequency）避免 class imbalance。
    可排除某些類別（如不存在的類別），這些類別的權重將設為 0。
    """
    exclude_classes = set(exclude_classes or [])
    class_sample_counts = np.bincount(y, minlength=num_classes)
    total_samples = len(y)

    weights = np.zeros(num_classes, dtype=np.float32)

    for cls in range(num_classes):
        if cls in exclude_classes:
            weights[cls] = 0.0
        else:
            count = class_sample_counts[cls]
            weights[cls] = total_samples / (count + 1e-6)

    # Normalize only non-excluded weights
    valid_mask = np.array([cls not in exclude_classes for cls in range(num_classes)])
    norm_sum = weights[valid_mask].sum()
    if norm_sum > 0:
        weights[valid_mask] /= norm_sum

    print("\n📊 Class Weights (normalized):")
    for i, w in enumerate(weights):
        status = " (excluded)" if i in exclude_classes else ""
        print(f"  - Class {i}: {w:.4f}{status}")
    
    return torch.tensor(weights, dtype=torch.float32)


### Training Function

In [14]:
def train_with_lora_ecg(model, output_size, criterion, optimizer,
                        X_train, y_train, X_val, y_val,
                        scheduler=None, num_epochs=10, batch_size=64,
                        model_saving_folder=None, model_name=None,
                        stop_signal_file=None, device=None):

    print("\n🚀 'train_with_lora_ecg' started.")
    start_time = time.time()

    device = device or auto_select_cuda_device()
    model_name = model_name or 'lora_model'
    model_saving_folder = model_saving_folder or './saved_models'

    if os.path.exists(model_saving_folder):
        shutil.rmtree(model_saving_folder)
        print(f"✅ Removed existing folder: {model_saving_folder}")
    os.makedirs(model_saving_folder, exist_ok=True)

    model.to(device)

    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.long).to(device)
    X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val = torch.tensor(y_val, dtype=torch.long).to(device)

    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size, shuffle=False)

    print("\n✅ Data Overview:")
    print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")

    best_results = []

    for epoch in range(num_epochs):
        if stop_signal_file and os.path.exists(stop_signal_file):
            print("\n🛑 Stop signal detected. Exiting training loop.")
            break

        model.train()
        epoch_loss = 0.0
        class_correct, class_total = {}, {}

        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * X_batch.size(0)
            compute_classwise_accuracy(outputs, y_batch, class_correct, class_total)

        train_loss = epoch_loss / len(train_loader.dataset)
        train_acc = {int(c): f"{(class_correct[c] / class_total[c]) * 100:.2f}%" if class_total[c] > 0 else "0.00%"
                     for c in sorted(class_total.keys())}

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        val_class_correct, val_class_total = {}, {}
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                outputs = model(X_batch)
                val_loss += criterion(outputs, y_batch).item() * X_batch.size(0)
                predictions = torch.argmax(outputs, dim=-1)
                val_correct += (predictions == y_batch).sum().item()
                val_total += y_batch.size(0)
                compute_classwise_accuracy(outputs, y_batch, val_class_correct, val_class_total)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct / val_total
        val_acc_cls = {int(c): f"{(val_class_correct[c] / val_class_total[c]) * 100:.2f}%" if val_class_total[c] > 0 else "0.00%"
                       for c in sorted(val_class_total.keys())}

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.6f}, Train-Class-Acc: {train_acc}")
        print(f"Val Loss: {val_loss:.6f}, Val Acc: {val_acc * 100:.2f}%, Val-Class-Acc: {val_acc_cls}, LR: {optimizer.param_groups[0]['lr']:.6f}")

        model_path = os.path.join(model_saving_folder, f"{model_name}_epoch_{epoch+1}.pth")
        current = {
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'val_accuracy': val_acc,
            'train_classwise_accuracy': train_acc,
            'val_classwise_accuracy': val_acc_cls,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'learning_rate': optimizer.param_groups[0]['lr'],
            'model_path': model_path
        }

        if len(best_results) < 5 or val_acc > best_results[-1]['val_accuracy']:
            if len(best_results) == 5:
                to_remove = best_results.pop()
                if os.path.exists(to_remove['model_path']):
                    os.remove(to_remove['model_path'])
                    print(f"🗑 Removed: {to_remove['model_path']}")
            best_results.append(current)
            best_results.sort(key=lambda x: (x['val_accuracy'], x['epoch']), reverse=True)
            torch.save(current, model_path)
            print(f"✅ Saved model: {model_path}")

        if scheduler:
            scheduler.step(val_loss)

    training_time = time.time() - start_time
    total_params, param_size_MB = get_model_parameter_info(model)

    if best_results:
        best = best_results[0]
        best_model_path = os.path.join(model_saving_folder, f"{model_name}_best.pth")
        torch.save(best, best_model_path)
        print(f"\n🏆 Best model saved as: {best_model_path} (Val Accuracy: {best['val_accuracy'] * 100:.2f}%)")

    final_model_path = os.path.join(model_saving_folder, f"{model_name}_final.pth")
    torch.save(current, final_model_path)
    print(f"\n📌 Final model saved as: {final_model_path}")

    print("\n🎯 Top 5 Best Models:")
    for res in best_results:
        print(f"Epoch {res['epoch']}, Train Loss: {res['train_loss']:.6f}, Train-Acc: {res['train_classwise_accuracy']},\n"
              f"Val Loss: {res['val_loss']:.6f}, Val Acc: {res['val_accuracy']*100:.2f}%, Val-Acc: {res['val_classwise_accuracy']},"
              f" Model Path: {res['model_path']}")

    match = re.search(r'Period_(\d+)', model_saving_folder)
    period_label = match.group(1) if match else "?"
    model_name_str = model.__class__.__name__

    print(f"""
---
### Period {period_label}
+ ##### Total training time: {training_time:.2f} seconds
+ ##### Model: {model_name_str}
+ ##### Training and saving in *'{model_saving_folder}'*
+ ##### Best Epoch: {best['epoch']}
#### __Val Accuracy: {best['val_accuracy'] * 100:.2f}%__
#### __Val-Class-Acc: {best['val_classwise_accuracy']}__
#### __Total Parameters: {total_params:,}__
#### __Model Size (float32): {param_size_MB:.2f} MB__
""".strip())

    del X_train, y_train, X_val, y_val, train_loader, val_loader, current, outputs, predictions
    torch.cuda.empty_cache()
    gc.collect()


## __Training__

### Period 1 Summary

In [17]:
def display_model_summary_with_params(model_folder, model_filename="ResNet18_big_inplane_1D_best.pth", input_channels=12, output_size=10):
    model_path = os.path.join(model_folder, model_filename)

    if not os.path.exists(model_path):
        print(f"❌ File not found: {model_path}")
        return

    checkpoint = torch.load(model_path, map_location='cpu')

    # === 還原模型並載入參數 ===
    model = ResNet18_1D_LoRA(input_channels=input_channels, output_size=output_size)
    model.load_state_dict(checkpoint["model_state_dict"])
    total_params, param_size_MB = get_model_parameter_info(model)

    # === 顯示摘要 ===
    epoch = checkpoint.get("epoch", "?")
    train_loss = checkpoint.get("train_loss", "?")
    val_loss = checkpoint.get("val_loss", "?")
    val_acc = checkpoint.get("val_accuracy", "?")
    train_acc_dict = checkpoint.get("train_classwise_accuracy", {})
    val_acc_dict = checkpoint.get("val_classwise_accuracy", {})
    lr = checkpoint.get("learning_rate", "?")
    stored_path = checkpoint.get("model_path", "N/A")
    print(f"Model Architecture:")
    print(model)
    print(f"\n📦 Model Summary from: {model_path}")
    print(f"📌 Epoch: {epoch}")
    print(f"🧮 Train Loss: {train_loss:.6f}" if isinstance(train_loss, float) else f"🧮 Train Loss: {train_loss}")
    print(f"🎯 Val Loss: {val_loss:.6f}" if isinstance(val_loss, float) else f"🎯 Val Loss: {val_loss}")
    print(f"✅ Val Accuracy: {val_acc*100:.2f}%" if isinstance(val_acc, float) else f"✅ Val Accuracy: {val_acc}")
    print(f"📎 Learning Rate: {lr}")
    print(f"📁 Stored Model Path: {stored_path}")
    print(f"🧠 Total Parameters: {total_params:,}")
    print(f"📏 Model Size (float32): {param_size_MB:.2f} MB")

    print("\n📊 Train Class-wise Accuracy:")
    for c, acc in train_acc_dict.items():
        print(f"  └─ Class {c:<2}: {acc}")

    print("\n📊 Val Class-wise Accuracy:")
    for c, acc in val_acc_dict.items():
        print(f"  └─ Class {c:<2}: {acc}")

    print("\n---\n### Period 1 Summary (Markdown Format)")
    print(f"+ **Epoch:** {epoch}")
    print(f"+ **Train Loss:** {train_loss}")
    print(f"+ **Val Loss:** {val_loss}")
    print(f"+ **Val Accuracy:** {val_acc*100:.2f}%" if isinstance(val_acc, float) else f"+ **Val Accuracy:** {val_acc}")
    print(f"+ **Learning Rate:** {lr}")
    print(f"+ **Stored Model Path:** `{stored_path}`")
    print(f"+ **Total Parameters:** {total_params:,}")
    print(f"+ **Model Size (float32):** {param_size_MB:.2f} MB")
    print(f"+ **Train-Class-Acc:** {train_acc_dict}")
    print(f"+ **Val-Class-Acc:** {val_acc_dict}")
    print("---")

# Example call:
display_model_summary_with_params(
    model_folder=os.path.join("Class_Incremental_CL", "CPSC_CIL", "ResNet18_Selection", "ResNet18_big_inplane_v1"),
    input_channels=12,
    output_size=2
)

Model Architecture:
ResNet18_1D_LoRA(
  (conv1): Conv1d(12, 64, kernel_size=(15,), stride=(2,), padding=(7,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock1d_LoRA(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock1d_LoRA(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu):

/tmp/ipykernel_763947/2299765635.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location='cpu')


### Period 2

#### v1: Trainable parameter未排除`lora_adapter.conv`的版本

In [18]:
# ================================
# 📌 Period 2: Standard LoRA Training (ECG)
# ================================
period = 2

# ==== Paths ====
stop_signal_file = os.path.join(BASE_DIR, "stop_training.txt")
model_saving_folder = os.path.join(BASE_DIR, "Trained_models", "Standard_LoRA_CIL_v1", f"Period_{period}")
ensure_folder(model_saving_folder)

# ==== Load Period 2 Data ====
X_train = np.load(os.path.join(save_dir, f"X_train_p{period}.npy"))
y_train = np.load(os.path.join(save_dir, f"y_train_p{period}.npy"))
X_val   = np.load(os.path.join(save_dir, f"X_test_p{period}.npy"))
y_val   = np.load(os.path.join(save_dir, f"y_test_p{period}.npy"))

# ==== Device ====
device = auto_select_cuda_device()

# ==== Model Configuration ====
input_channels = X_train.shape[2]  # ECG 12-lead
output_size = len(np.unique(y_train))  # 新增類別數
model = ResNet18_1D_LoRA(input_channels=input_channels, output_size=output_size, lora_rank=4).to(device)

# ==== Load Period 1 Best Model Weights (excluding FC but keeping internal layers) ====
prev_model_path = os.path.join(BASE_DIR, "ResNet18_Selection", "ResNet18_big_inplane_v1", "ResNet18_big_inplane_1D_best.pth")
prev_checkpoint = torch.load(prev_model_path, map_location=device)
prev_state_dict = prev_checkpoint["model_state_dict"]

model_dict = model.state_dict()
filtered_state_dict = {
    k: v for k, v in prev_state_dict.items()
    if k in model_dict and model_dict[k].shape == v.shape and not k.startswith("fc")
}
model.load_state_dict(filtered_state_dict, strict=False)
for k in model_dict:
    if k not in filtered_state_dict:
        print(f"🔍 Not loaded: {k}, shape={model_dict[k].shape}")
print("✅ Loaded Period 1 weights (excluding final FC layer)")

# ==== Initialize LoRA Adapters ====
model.init_lora()

# ==== Optimizer / Scheduler ====
learning_rate = 1e-3
weight_decay = 1e-5
num_epochs = 200
batch_size = 64
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.get_trainable_parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Start Training ====
train_with_lora_ecg(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name="ResNet18_1D_LoRA",
    stop_signal_file=stop_signal_file,
    device=device
)

# ================================
# ✅ Cleanup
# ================================
del X_train, y_train, X_val, y_val
del prev_model_path, prev_checkpoint, prev_state_dict, filtered_state_dict
del model, criterion, optimizer, scheduler
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 1202 MiB
    - Device Name    : NVIDIA RTX A6000


/tmp/ipykernel_763947/3538613332.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_model_path, map_location=device)


🔍 Not loaded: fc.weight, shape=torch.Size([4, 1024])
🔍 Not loaded: fc.bias, shape=torch.Size([4])
✅ Loaded Period 1 weights (excluding final FC layer)
✅ LoRA adapters initialized for 8 conv2 layers
📊 Parameter Statistics:
  - Total parameters: 3,889,796
  - Trainable parameters: 2,123,780 (54.60%)
    - LoRA parameters: 2,119,680 (54.49%)
    - FC parameters: 4,100 (0.11%)
  - Frozen parameters: 1,766,016 (45.40%)
🧠 Trainable parameter names:
  ✅ layer1.0.lora_adapter.lora_A (LoRA)
  ✅ layer1.0.lora_adapter.lora_B (LoRA)
  ✅ layer1.0.lora_adapter.conv.weight (LoRA)
  ✅ layer1.1.lora_adapter.lora_A (LoRA)
  ✅ layer1.1.lora_adapter.lora_B (LoRA)
  ✅ layer1.1.lora_adapter.conv.weight (LoRA)
  ✅ layer2.0.lora_adapter.lora_A (LoRA)
  ✅ layer2.0.lora_adapter.lora_B (LoRA)
  ✅ layer2.0.lora_adapter.conv.weight (LoRA)
  ✅ layer2.1.lora_adapter.lora_A (LoRA)
  ✅ layer2.1.lora_adapter.lora_B (LoRA)
  ✅ layer2.1.lora_adapter.conv.weight (LoRA)
  ✅ layer3.0.lora_adapter.lora_A (LoRA)
  ✅ layer3.0.

#### v2: `lora_adapter.conv`未被載入的版本 (但這邊並不影響，因為period 1沒有LoRA)

In [20]:
# ================================
# 📌 Period 2: Standard LoRA Training (ECG)
# ================================
period = 2

# ==== Paths ====
stop_signal_file = os.path.join(BASE_DIR, "stop_training.txt")
model_saving_folder = os.path.join(BASE_DIR, "Trained_models", "Standard_LoRA_CIL_v2", f"Period_{period}")
ensure_folder(model_saving_folder)

# ==== Load Period 2 Data ====
X_train = np.load(os.path.join(save_dir, f"X_train_p{period}.npy"))
y_train = np.load(os.path.join(save_dir, f"y_train_p{period}.npy"))
X_val   = np.load(os.path.join(save_dir, f"X_test_p{period}.npy"))
y_val   = np.load(os.path.join(save_dir, f"y_test_p{period}.npy"))

# ==== Device ====
device = auto_select_cuda_device()

# ==== Model Configuration ====
input_channels = X_train.shape[2]  # ECG 12-lead
output_size = len(np.unique(y_train))  # 新增類別數
model = ResNet18_1D_LoRA(input_channels=input_channels, output_size=output_size, lora_rank=4).to(device)

# ==== Load Period 1 Best Model Weights (excluding FC but keeping internal layers) ====
prev_model_path = os.path.join(BASE_DIR, "ResNet18_Selection", "ResNet18_big_inplane_v1", "ResNet18_big_inplane_1D_best.pth")
prev_checkpoint = torch.load(prev_model_path, map_location=device)
prev_state_dict = prev_checkpoint["model_state_dict"]

model_dict = model.state_dict()
filtered_state_dict = {
    k: v for k, v in prev_state_dict.items()
    if k in model_dict and model_dict[k].shape == v.shape and not k.startswith("fc")
}
model.load_state_dict(filtered_state_dict, strict=False)
for k in model_dict:
    if k not in filtered_state_dict:
        print(f"🔍 Not loaded: {k}, shape={model_dict[k].shape}")
print("✅ Loaded Period 1 weights (excluding final FC layer)")

# ==== Initialize LoRA Adapters ====
model.init_lora()

# ==== Optimizer / Scheduler ====
learning_rate = 1e-3
weight_decay = 1e-5
num_epochs = 200
batch_size = 64
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.get_trainable_parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Start Training ====
train_with_lora_ecg(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name="ResNet18_1D_LoRA",
    stop_signal_file=stop_signal_file,
    device=device
)

# ================================
# ✅ Cleanup
# ================================
del X_train, y_train, X_val, y_val
del prev_model_path, prev_checkpoint, prev_state_dict, filtered_state_dict
del model, criterion, optimizer, scheduler
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 1409 MiB
    - Device Name    : NVIDIA RTX A6000


/tmp/ipykernel_763947/3172304596.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_model_path, map_location=device)


🔍 Not loaded: fc.weight, shape=torch.Size([4, 1024])
🔍 Not loaded: fc.bias, shape=torch.Size([4])
✅ Loaded Period 1 weights (excluding final FC layer)
✅ LoRA adapters initialized for 8 conv2 layers
📊 Parameter Statistics:
  - Total parameters: 3,889,796
  - Trainable parameters: 34,820 (0.90%)
    - LoRA parameters: 30,720 (0.79%)
    - FC parameters: 4,100 (0.11%)
  - Frozen parameters: 3,854,976 (99.10%)
🧠 Trainable parameter names:
  ✅ layer1.0.lora_adapter.lora_A (LoRA)
  ✅ layer1.0.lora_adapter.lora_B (LoRA)
  ✅ layer1.1.lora_adapter.lora_A (LoRA)
  ✅ layer1.1.lora_adapter.lora_B (LoRA)
  ✅ layer2.0.lora_adapter.lora_A (LoRA)
  ✅ layer2.0.lora_adapter.lora_B (LoRA)
  ✅ layer2.1.lora_adapter.lora_A (LoRA)
  ✅ layer2.1.lora_adapter.lora_B (LoRA)
  ✅ layer3.0.lora_adapter.lora_A (LoRA)
  ✅ layer3.0.lora_adapter.lora_B (LoRA)
  ✅ layer3.1.lora_adapter.lora_A (LoRA)
  ✅ layer3.1.lora_adapter.lora_B (LoRA)
  ✅ layer4.0.lora_adapter.lora_A (LoRA)
  ✅ layer4.0.lora_adapter.lora_B (LoRA)
 

### Period 3

#### v2: `lora_adapter.conv`未被載入的版本

In [21]:
# ================================
# 📌 Period 3: Standard LoRA Training (ECG)
# ================================
period = 3

# ==== Paths ====
stop_signal_file = os.path.join(BASE_DIR, "stop_training.txt")
model_saving_folder = os.path.join(BASE_DIR, "Trained_models", "Standard_LoRA_CIL_v2", f"Period_{period}")
ensure_folder(model_saving_folder)

# ==== Load Period 3 Data ====
X_train = np.load(os.path.join(save_dir, f"X_train_p{period}.npy"))
y_train = np.load(os.path.join(save_dir, f"y_train_p{period}.npy"))
X_val   = np.load(os.path.join(save_dir, f"X_test_p{period}.npy"))
y_val   = np.load(os.path.join(save_dir, f"y_test_p{period}.npy"))

# ==== Device ====
device = auto_select_cuda_device()

# ==== Model Configuration ====
input_channels = X_train.shape[2]  # ECG 12-lead
output_size = len(np.unique(y_train))  # 新增類別數
model = ResNet18_1D_LoRA(input_channels=input_channels, output_size=output_size, lora_rank=4).to(device)

# ==== Initialize LoRA Adapters FIRST ====
model.init_lora()

# ==== Load Period 2 Best Model Weights (excluding FC & LoRA) ====
prev_model_path = os.path.join(BASE_DIR, "Trained_models", "Standard_LoRA_CIL_v2", f"Period_{period - 1}", "ResNet18_1D_LoRA_best.pth")
prev_checkpoint = torch.load(prev_model_path, map_location=device)
prev_state_dict = prev_checkpoint["model_state_dict"]

model_dict = model.state_dict()
filtered_state_dict = {
    k: v for k, v in prev_state_dict.items()
    if k in model_dict and model_dict[k].shape == v.shape and not (k.startswith("fc") or "lora_adapter" in k)
}
model.load_state_dict(filtered_state_dict, strict=False)
for k in model_dict:
    if k not in filtered_state_dict:
        print(f"🔍 Not loaded: {k}, shape={model_dict[k].shape}")
print("✅ Loaded Period 2 weights (excluding FC & LoRA)")

# ==== Optimizer / Scheduler ====
learning_rate = 1e-3
weight_decay = 1e-5
num_epochs = 200
batch_size = 64
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.get_trainable_parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Start Training ====
train_with_lora_ecg(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name="ResNet18_1D_LoRA",
    stop_signal_file=stop_signal_file,
    device=device
)

# ================================
# ✅ Cleanup
# ================================
del X_train, y_train, X_val, y_val
del prev_model_path, prev_checkpoint, prev_state_dict, filtered_state_dict
del model, criterion, optimizer, scheduler
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 1597 MiB
    - Device Name    : NVIDIA RTX A6000
✅ LoRA adapters initialized for 8 conv2 layers
🔍 Not loaded: layer1.0.lora_adapter.lora_A, shape=torch.Size([64, 4])
🔍 Not loaded: layer1.0.lora_adapter.lora_B, shape=torch.Size([4, 192])
🔍 Not loaded: layer1.0.lora_adapter.conv.weight, shape=torch.Size([64, 64, 3])
🔍 Not loaded: layer1.1.lora_adapter.lora_A, shape=torch.Size([64, 4])
🔍 Not loaded: layer1.1.lora_adapter.lora_B, shape=torch.Size([4, 192])
🔍 Not loaded: layer1.1.lora_adapter.conv.weight, shape=torch.Size([64, 64, 3])
🔍 Not loaded: layer2.0.lora_adapter.lora_A, shape=torch.Size([128, 4])
🔍 Not loaded: layer2.0.lora_adapter.lora_B, shape=torch.Size([4, 384])
🔍 Not loaded: layer2.0.lora_adapter.conv.weight, shape=torch.Size([128, 128, 3])
🔍 Not loaded: layer2.1.lora_adapter.lora_A, shape=torch.Size([128, 4])
🔍 Not loaded: layer2.1.lora_adapter.lora_B, shape=torch.Size([4, 384])
🔍 Not loaded: layer2.

/tmp/ipykernel_763947/1785557546.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_model_path, map_location=device)



✅ Data Overview:
X_train: torch.Size([5120, 5000, 12]), y_train: torch.Size([5120])
X_val: torch.Size([1281, 5000, 12]), y_val: torch.Size([1281])
Epoch 1/200, Train Loss: 57.088251, Train-Class-Acc: {0: '39.65%', 1: '28.42%', 2: '18.37%', 3: '27.25%', 4: '12.66%', 5: '43.20%'}
Val Loss: 13.715727, Val Acc: 58.16%, Val-Class-Acc: {0: '63.04%', 1: '58.21%', 2: '22.92%', 3: '45.49%', 4: '7.50%', 5: '85.93%'}, LR: 0.001000
✅ Saved model: /mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL/Trained_models/Standard_LoRA_CIL_v2/Period_3/ResNet18_1D_LoRA_epoch_1.pth
Epoch 2/200, Train Loss: 15.742894, Train-Class-Acc: {0: '61.44%', 1: '46.75%', 2: '30.68%', 3: '38.42%', 4: '20.89%', 5: '71.00%'}
Val Loss: 6.599931, Val Acc: 66.82%, Val-Class-Acc: {0: '89.13%', 1: '60.90%', 2: '48.61%', 3: '51.23%', 4: '32.50%', 5: '83.83%'}, LR: 0.001000
✅ Saved model: /mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL/Trained_models/Standard_LoRA

#### v3: 修改載入參數邏輯，`lora_adapter.conv`也要載入

In [22]:
# ================================
# 📌 Period 3: Standard LoRA Training (ECG)
# ================================
period = 3

# ==== Paths ====
stop_signal_file = os.path.join(BASE_DIR, "stop_training.txt")
model_saving_folder = os.path.join(BASE_DIR, "Trained_models", "Standard_LoRA_CIL_v3", f"Period_{period}")
ensure_folder(model_saving_folder)

# ==== Load Period 3 Data ====
X_train = np.load(os.path.join(save_dir, f"X_train_p{period}.npy"))
y_train = np.load(os.path.join(save_dir, f"y_train_p{period}.npy"))
X_val   = np.load(os.path.join(save_dir, f"X_test_p{period}.npy"))
y_val   = np.load(os.path.join(save_dir, f"y_test_p{period}.npy"))

# ==== Device ====
device = auto_select_cuda_device()

# ==== Model Configuration ====
input_channels = X_train.shape[2]  # ECG 12-lead
output_size = len(np.unique(y_train))  # 新增類別數
model = ResNet18_1D_LoRA(input_channels=input_channels, output_size=output_size, lora_rank=4).to(device)

# ==== Initialize LoRA Adapters FIRST ====
model.init_lora()

# ==== Load Period 2 Best Model Weights (excluding FC & LoRA) ====
prev_model_path = os.path.join(BASE_DIR, "Trained_models", "Standard_LoRA_CIL_v2", f"Period_{period - 1}", "ResNet18_1D_LoRA_best.pth")
prev_checkpoint = torch.load(prev_model_path, map_location=device)
prev_state_dict = prev_checkpoint["model_state_dict"]

model_dict = model.state_dict()
# filtered_state_dict = {
#     k: v for k, v in prev_state_dict.items()
#     if k in model_dict and model_dict[k].shape == v.shape and not (k.startswith("fc") or "lora_adapter" in k)
# }
filtered_state_dict = {
    k: v for k, v in prev_state_dict.items()
    if k in model_dict and model_dict[k].shape == v.shape and not (
        k.startswith("fc") or "lora_adapter.lora_A" in k or "lora_adapter.lora_B" in k
    )
}
model.load_state_dict(filtered_state_dict, strict=False)
for k in model_dict:
    if k not in filtered_state_dict:
        print(f"🔍 Not loaded: {k}, shape={model_dict[k].shape}")
print("✅ Loaded Period 2 weights (excluding FC & LoRA)")

# ==== Optimizer / Scheduler ====
learning_rate = 1e-3
weight_decay = 1e-5
num_epochs = 200
batch_size = 64
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.get_trainable_parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Start Training ====
train_with_lora_ecg(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name="ResNet18_1D_LoRA",
    stop_signal_file=stop_signal_file,
    device=device
)

# ================================
# ✅ Cleanup
# ================================
del X_train, y_train, X_val, y_val
del prev_model_path, prev_checkpoint, prev_state_dict, filtered_state_dict
del model, criterion, optimizer, scheduler
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 1603 MiB
    - Device Name    : NVIDIA RTX A6000
✅ LoRA adapters initialized for 8 conv2 layers
🔍 Not loaded: layer1.0.lora_adapter.lora_A, shape=torch.Size([64, 4])
🔍 Not loaded: layer1.0.lora_adapter.lora_B, shape=torch.Size([4, 192])
🔍 Not loaded: layer1.1.lora_adapter.lora_A, shape=torch.Size([64, 4])
🔍 Not loaded: layer1.1.lora_adapter.lora_B, shape=torch.Size([4, 192])
🔍 Not loaded: layer2.0.lora_adapter.lora_A, shape=torch.Size([128, 4])
🔍 Not loaded: layer2.0.lora_adapter.lora_B, shape=torch.Size([4, 384])
🔍 Not loaded: layer2.1.lora_adapter.lora_A, shape=torch.Size([128, 4])
🔍 Not loaded: layer2.1.lora_adapter.lora_B, shape=torch.Size([4, 384])
🔍 Not loaded: layer3.0.lora_adapter.lora_A, shape=torch.Size([256, 4])
🔍 Not loaded: layer3.0.lora_adapter.lora_B, shape=torch.Size([4, 768])
🔍 Not loaded: layer3.1.lora_adapter.lora_A, shape=torch.Size([256, 4])
🔍 Not loaded: layer3.1.lora_adapter.lora_B, sha

/tmp/ipykernel_763947/394820996.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_model_path, map_location=device)



✅ Data Overview:
X_train: torch.Size([5120, 5000, 12]), y_train: torch.Size([5120])
X_val: torch.Size([1281, 5000, 12]), y_val: torch.Size([1281])
Epoch 1/200, Train Loss: 39.377799, Train-Class-Acc: {0: '47.00%', 1: '37.10%', 2: '16.29%', 3: '25.20%', 4: '5.70%', 5: '52.09%'}
Val Loss: 11.803610, Val Acc: 61.20%, Val-Class-Acc: {0: '69.57%', 1: '54.93%', 2: '36.11%', 3: '50.82%', 4: '0.00%', 5: '88.62%'}, LR: 0.001000
✅ Saved model: /mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL/Trained_models/Standard_LoRA_CIL_v3/Period_3/ResNet18_1D_LoRA_epoch_1.pth
Epoch 2/200, Train Loss: 15.199642, Train-Class-Acc: {0: '63.35%', 1: '47.49%', 2: '35.01%', 3: '37.60%', 4: '10.13%', 5: '72.50%'}
Val Loss: 7.343332, Val Acc: 67.14%, Val-Class-Acc: {0: '79.35%', 1: '60.00%', 2: '56.25%', 3: '53.28%', 4: '22.50%', 5: '87.72%'}, LR: 0.001000
✅ Saved model: /mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL/Trained_models/Standard_LoRA_

### Period 4

In [23]:
# ================================
# 📌 Period 4: Standard LoRA Training (ECG)
# ================================
period = 4

# ==== Paths ====
stop_signal_file = os.path.join(BASE_DIR, "stop_training.txt")
model_saving_folder = os.path.join(BASE_DIR, "Trained_models", "Standard_LoRA_CIL_v3", f"Period_{period}")
ensure_folder(model_saving_folder)

# ==== Load Period 4 Data ====
X_train = np.load(os.path.join(save_dir, f"X_train_p{period}.npy"))
y_train = np.load(os.path.join(save_dir, f"y_train_p{period}.npy"))
X_val   = np.load(os.path.join(save_dir, f"X_test_p{period}.npy"))
y_val   = np.load(os.path.join(save_dir, f"y_test_p{period}.npy"))

# ==== Device ====
device = auto_select_cuda_device()

# ==== Model Configuration ====
input_channels = X_train.shape[2]  # ECG 12-lead
output_size = int(np.max(y_train)) + 1  # e.g., max=9 → output_size=10
model = ResNet18_1D_LoRA(input_channels=input_channels, output_size=output_size, lora_rank=4).to(device)

# ==== Initialize LoRA Adapters FIRST ====
model.init_lora()

# ==== Load Period 3 Best Model Weights (excluding FC & LoRA A/B) ====
prev_model_path = os.path.join(BASE_DIR, "Trained_models", "Standard_LoRA_CIL_v3", f"Period_{period - 1}", "ResNet18_1D_LoRA_best.pth")
prev_checkpoint = torch.load(prev_model_path, map_location=device)
prev_state_dict = prev_checkpoint["model_state_dict"]

model_dict = model.state_dict()
filtered_state_dict = {
    k: v for k, v in prev_state_dict.items()
    if k in model_dict and model_dict[k].shape == v.shape and not (
        k.startswith("fc") or "lora_adapter.lora_A" in k or "lora_adapter.lora_B" in k
    )
}
model.load_state_dict(filtered_state_dict, strict=False)
for k in model_dict:
    if k not in filtered_state_dict:
        print(f"🔍 Not loaded: {k}, shape={model_dict[k].shape}")
print("✅ Loaded Period 3 weights (excluding FC & LoRA A/B)")

# ==== Optimizer / Scheduler ====
learning_rate = 1e-3
weight_decay = 1e-5
num_epochs = 200
batch_size = 64
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.get_trainable_parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Start Training ====
train_with_lora_ecg(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name="ResNet18_1D_LoRA",
    stop_signal_file=stop_signal_file,
    device=device
)

# ================================
# ✅ Cleanup
# ================================
del X_train, y_train, X_val, y_val
del prev_model_path, prev_checkpoint, prev_state_dict, filtered_state_dict
del model, criterion, optimizer, scheduler
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 1607 MiB
    - Device Name    : NVIDIA RTX A6000
✅ LoRA adapters initialized for 8 conv2 layers
🔍 Not loaded: layer1.0.lora_adapter.lora_A, shape=torch.Size([64, 4])
🔍 Not loaded: layer1.0.lora_adapter.lora_B, shape=torch.Size([4, 192])
🔍 Not loaded: layer1.1.lora_adapter.lora_A, shape=torch.Size([64, 4])
🔍 Not loaded: layer1.1.lora_adapter.lora_B, shape=torch.Size([4, 192])
🔍 Not loaded: layer2.0.lora_adapter.lora_A, shape=torch.Size([128, 4])
🔍 Not loaded: layer2.0.lora_adapter.lora_B, shape=torch.Size([4, 384])
🔍 Not loaded: layer2.1.lora_adapter.lora_A, shape=torch.Size([128, 4])
🔍 Not loaded: layer2.1.lora_adapter.lora_B, shape=torch.Size([4, 384])
🔍 Not loaded: layer3.0.lora_adapter.lora_A, shape=torch.Size([256, 4])
🔍 Not loaded: layer3.0.lora_adapter.lora_B, shape=torch.Size([4, 768])
🔍 Not loaded: layer3.1.lora_adapter.lora_A, shape=torch.Size([256, 4])
🔍 Not loaded: layer3.1.lora_adapter.lora_B, sha

/tmp/ipykernel_763947/3463635584.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_model_path, map_location=device)



✅ Data Overview:
X_train: torch.Size([5493, 5000, 12]), y_train: torch.Size([5493])
X_val: torch.Size([1374, 5000, 12]), y_val: torch.Size([1374])
Epoch 1/200, Train Loss: 33.972773, Train-Class-Acc: {0: '38.42%', 2: '10.40%', 3: '25.72%', 4: '5.06%', 5: '56.77%', 6: '10.37%', 7: '20.56%', 8: '24.52%', 9: '6.08%'}
Val Loss: 11.989978, Val Acc: 53.42%, Val-Class-Acc: {0: '73.37%', 2: '6.94%', 3: '40.57%', 4: '7.50%', 5: '88.06%', 6: '8.33%', 7: '69.60%', 8: '61.15%', 9: '0.00%'}, LR: 0.001000
✅ Saved model: /mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL/Trained_models/Standard_LoRA_CIL_v3/Period_4/ResNet18_1D_LoRA_epoch_1.pth
Epoch 2/200, Train Loss: 13.480584, Train-Class-Acc: {0: '57.63%', 2: '23.57%', 3: '37.19%', 4: '23.42%', 5: '70.91%', 6: '10.14%', 7: '35.93%', 8: '32.48%', 9: '8.11%'}
Val Loss: 6.176110, Val Acc: 57.35%, Val-Class-Acc: {0: '72.28%', 2: '39.58%', 3: '40.98%', 4: '25.00%', 5: '85.67%', 6: '11.11%', 7: '64.80%', 8: '68.15%', 9: 

##  Compute FWT

In [15]:
def compute_fwt_ecg(previous_model, init_model, X_val, y_val, known_classes, batch_size=64):
    """
    FWT computation for ECG-style inputs with 1D CNN (e.g., ResNet18_1D).
    X_val: shape [B, T, C]  (e.g., [N, 5000, 12])
    y_val: shape [B]        (e.g., [N])
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    previous_model.to(device).eval()
    init_model.to(device).eval()

    # 只選取 known classes
    mask = np.isin(y_val, known_classes)
    X_known = X_val[mask]
    y_known = y_val[mask]

    if len(y_known) == 0:
        print(f"⚠️ No validation samples for known classes {known_classes}.")
        return None, None, None

    print(f"📋 Total samples for known classes {known_classes}: {len(y_known)}")

    dataset = TensorDataset(
        torch.tensor(X_known, dtype=torch.float32),
        torch.tensor(y_known, dtype=torch.long)
    )
    loader = DataLoader(dataset, batch_size=batch_size)

    correct_prev, correct_init, total = 0, 0, 0

    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)

            out_prev = previous_model(xb)  # [B, C]
            out_init = init_model(xb)

            preds_prev = torch.argmax(out_prev, dim=-1)
            preds_init = torch.argmax(out_init, dim=-1)

            correct_prev += (preds_prev == yb).sum().item()
            correct_init += (preds_init == yb).sum().item()
            total += yb.size(0)

    acc_prev = 100 * correct_prev / total
    acc_init = 100 * correct_init / total
    fwt_value = acc_prev - acc_init

    print(f"\n### 🔍 FWT Debug Info:")
    print(f"- Total evaluated samples: {total}")
    print(f"- Accuracy by previous model: {acc_prev:.2f}%")
    print(f"- Accuracy by init model:     {acc_init:.2f}%")
    print(f"- FWT = Acc_prev - Acc_init = {fwt_value:.2f}%")

    return fwt_value, acc_prev, acc_init


### Period 2

In [16]:
# ================================
# 📌 FWT - Period 2 (Standard LoRA - ECG)
# ================================
period = 2

# === Load Validation Data ===
X_val = np.load(os.path.join(save_dir, f"X_test_p{period}.npy"))
y_val = np.load(os.path.join(save_dir, f"y_test_p{period}.npy"))

device = auto_select_cuda_device()
input_channels = X_val.shape[2]
output_size_prev = 2  # Period 1 output classes
known_classes = [0, 1]  # Period 2 中的穩定舊類別

# === Load Period 1 Baseline Model ===
prev_model_path = os.path.join(BASE_DIR, "ResNet18_Selection", "ResNet18_big_inplane_v1", "ResNet18_big_inplane_1D_best.pth")
checkpoint = torch.load(prev_model_path, map_location=device)

frozen_model = ResNet18_1D_LoRA(input_channels=input_channels, output_size=output_size_prev, lora_rank=4).to(device)
frozen_model.load_state_dict(checkpoint["model_state_dict"])
frozen_model.output_size = output_size_prev
frozen_model.eval()
del checkpoint

# === Init Model ===
init_model = ResNet18_1D_LoRA(input_channels=input_channels, output_size=output_size_prev, lora_rank=4).to(device)
init_model.output_size = output_size_prev

# === Tensor Conversion ===
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# === Run FWT Evaluation ===
fwt, acc_prev, acc_init = compute_fwt_ecg(frozen_model, init_model, X_val_tensor, y_val_tensor, known_classes)

print(f"\n### Period 2:")
print(f"- FWT (Standard LoRA ECG Period 2, old classes {known_classes}): {fwt:.2f}%")
print(f"- Accuracy by previous model: {acc_prev:.2f}%")
print(f"- Accuracy by init model:     {acc_init:.2f}%")


🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 3281 MiB
    - Device Name    : NVIDIA RTX A6000


/tmp/ipykernel_149835/2406496834.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(prev_model_path, map_location=device)


📋 Total samples for known classes [0, 1]: 428


/tmp/ipykernel_149835/1383618855.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(X_known, dtype=torch.float32),
/tmp/ipykernel_149835/1383618855.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y_known, dtype=torch.long)



### 🔍 FWT Debug Info:
- Total evaluated samples: 428
- Accuracy by previous model: 89.25%
- Accuracy by init model:     49.53%
- FWT = Acc_prev - Acc_init = 39.72%

### Period 2:
- FWT (Standard LoRA ECG Period 2, old classes [0, 1]): 39.72%
- Accuracy by previous model: 89.25%
- Accuracy by init model:     49.53%


### Period 3

In [17]:
# ================================
# 📌 FWT - Period 3 (Standard LoRA - ECG)
# ================================
period = 3

# === Load Validation Data ===
X_val = np.load(os.path.join(save_dir, f"X_test_p{period}.npy"))
y_val = np.load(os.path.join(save_dir, f"y_test_p{period}.npy"))

device = auto_select_cuda_device()
input_channels = X_val.shape[2]
output_size_prev = 4
known_classes = [0, 1, 2, 3]

# === Load Period 2 Best Model ===
prev_model_path = os.path.join(BASE_DIR, "Trained_models", "Standard_LoRA_CIL_v2", "Period_2", "ResNet18_1D_LoRA_best.pth")
checkpoint = torch.load(prev_model_path, map_location=device)
state_dict = checkpoint["model_state_dict"]
del checkpoint

# === Rebuild previous model and init LoRA ===
frozen_model = ResNet18_1D_LoRA(input_channels=input_channels, output_size=output_size_prev, lora_rank=4).to(device)
frozen_model.init_lora()
frozen_model.load_state_dict(state_dict, strict=False)
frozen_model.output_size = output_size_prev
frozen_model.eval()

# === Init model (untrained) ===
init_model = ResNet18_1D_LoRA(input_channels=input_channels, output_size=output_size_prev, lora_rank=4).to(device)
init_model.init_lora()
init_model.output_size = output_size_prev

# === Tensor Conversion ===
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# === Run FWT Evaluation ===
fwt, acc_prev, acc_init = compute_fwt_ecg(frozen_model, init_model, X_val_tensor, y_val_tensor, known_classes)

print(f"\n### Period 3:")
print(f"- FWT (Standard LoRA ECG Period 3, old classes {known_classes}): {fwt:.2f}%")
print(f"- Accuracy by previous model: {acc_prev:.2f}%")
print(f"- Accuracy by init model:     {acc_init:.2f}%")


🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 3848 MiB
    - Device Name    : NVIDIA RTX A6000
✅ LoRA adapters initialized for 8 conv2 layers
✅ LoRA adapters initialized for 8 conv2 layers
📋 Total samples for known classes [0, 1, 2, 3]: 907


/tmp/ipykernel_149835/3956697940.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(prev_model_path, map_location=device)
/tmp/ipykernel_149835/138


### 🔍 FWT Debug Info:
- Total evaluated samples: 907
- Accuracy by previous model: 87.32%
- Accuracy by init model:     32.30%
- FWT = Acc_prev - Acc_init = 55.02%

### Period 3:
- FWT (Standard LoRA ECG Period 3, old classes [0, 1, 2, 3]): 55.02%
- Accuracy by previous model: 87.32%
- Accuracy by init model:     32.30%


### Period 4

In [23]:
# ================================
# 📌 FWT - Period 4 (Standard LoRA - ECG)
# ================================
period = 4

# === Load Validation Data ===
X_val = np.load(os.path.join(save_dir, f"X_test_p{period}.npy"))
y_val = np.load(os.path.join(save_dir, f"y_test_p{period}.npy"))

device = auto_select_cuda_device()
input_channels = X_val.shape[2]
output_size_prev = 6
known_classes = [0, 1, 2, 3, 4, 5]

# === Load Period 2 Best Model ===
prev_model_path = os.path.join(BASE_DIR, "Trained_models", "Standard_LoRA_CIL_v3", "Period_3", "ResNet18_1D_LoRA_best.pth")
checkpoint = torch.load(prev_model_path, map_location=device)
state_dict = checkpoint["model_state_dict"]
del checkpoint

# === Rebuild previous model and init LoRA ===
frozen_model = ResNet18_1D_LoRA(input_channels=input_channels, output_size=output_size_prev, lora_rank=4).to(device)
frozen_model.init_lora()
frozen_model.load_state_dict(state_dict, strict=False)
frozen_model.output_size = output_size_prev
frozen_model.eval()

# === Init model (untrained) ===
init_model = ResNet18_1D_LoRA(input_channels=input_channels, output_size=output_size_prev, lora_rank=4).to(device)
init_model.init_lora()
init_model.output_size = output_size_prev

# === Tensor Conversion ===
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# === Run FWT Evaluation ===
fwt, acc_prev, acc_init = compute_fwt_ecg(frozen_model, init_model, X_val_tensor, y_val_tensor, known_classes)

print(f"\n### Period 4:")
print(f"- FWT (Standard LoRA ECG Period 4, old classes {known_classes}): {fwt:.2f}%")
print(f"- Accuracy by previous model: {acc_prev:.2f}%")
print(f"- Accuracy by init model:     {acc_init:.2f}%")


🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 20473 MiB
    - Device Name    : NVIDIA RTX A6000
✅ LoRA adapters initialized for 8 conv2 layers
✅ LoRA adapters initialized for 8 conv2 layers
📋 Total samples for known classes [0, 1, 2, 3, 4, 5]: 947


/tmp/ipykernel_149835/3776746818.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(prev_model_path, map_location=device)
/tmp/ipykernel_149835/138


### 🔍 FWT Debug Info:
- Total evaluated samples: 947
- Accuracy by previous model: 94.93%
- Accuracy by init model:     25.77%
- FWT = Acc_prev - Acc_init = 69.17%

### Period 4:
- FWT (Standard LoRA ECG Period 4, old classes [0, 1, 2, 3, 4, 5]): 69.17%
- Accuracy by previous model: 94.93%
- Accuracy by init model:     25.77%


## 📊 Summary: 

### ✔️ CPSC - Standard LoRA: Validation Summary

| Period | Training Time (s) | Validation Accuracy | Class-wise Accuracy                                                                 |
|--------|-------------------|---------------------|--------------------------------------------------------------------------------------|
| 1      | 134.66            | **88.86%**          | {0: 91.85%, 1: 85.87%}                                                              |
| 2      | 298.35            | **87.25%**          | {0: 85.33%, 1: 80.33%, 2: 85.42%, 3: 96.72%}                                        |
| 3      | 469.49            | **84.07%**          | {0: 74.46%, 1: 77.91%, 2: 86.11%, 3: 88.52%, 4: 82.50%, 5: 91.62%}                  |
| 4      | 506.13            | **81.08%**          | {0: 78.26%, 2: 82.64%, 3: 93.44%, 4: 75.00%, 5: 92.54%, 6: 37.04%, 7: 79.20%, 8: 82.80%, 9: 37.84%} |


### 🧠 Continual Learning Metrics

| Period | AA_old (%) | AA_new (%) | BWT (%) | FWT (%) | FWT Classes        | Prev. Model Acc | Init Model Acc |
|--------|------------|------------|---------|---------|---------------------|------------------|-----------------|
| 2      | 82.83%     | 91.07%     | -6.03%  | 39.72%  | [0, 1]              | 89.25%           | 49.53%          |
| 3      | 81.75%     | 87.06%     | -5.20%  | 55.02%  | [0, 1, 2, 3]        | 87.32%           | 32.30%          |
| 4      | 84.38%     | 59.22%     | -0.27%  | 69.17%  | [0, 1, 2, 3, 4, 5]  | 94.93%           | 25.77%          |


### 📦 Model Size per Period

| Period | Output Size | LoRA Added? | Total Params | Δ Params vs Prev | Δ % vs Prev | Model Size (float32) |
|--------|-------------|-------------|--------------|------------------|-------------|-----------------------|
| 1      | 2           | ✘ No        | 3,857,026    | —                | —           | 14.71 MB              |
| 2      | 4           | ✔ Yes       | 3,889,796    | +32,770          | +0.85%      | 14.84 MB              |
| 3      | 6           | ✘ No        | 3,891,846    | +2,050           | +0.05%      | 14.85 MB              |
| 4      | 10          | ✘ No        | 3,895,946    | +4,100           | +0.11%      | 14.86 MB              |

**📈 Model Growth Rate (MGR) = (3,895,946 - 3,857,026) / (3,857,026 × 3) ≈ +0.34%**

**📈 Max trainable ratio ≈ 1.05%**
